<a href="https://colab.research.google.com/github/rajeshbabua/AI-Generated-Media/blob/main/Edit_Faces_with_StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Edit Faces with StyleGAN2
###I've created this colab notebook upon popular request on my YT videos & on reddit. With it you can search for any person in the output space of StyleGAN2 (thx NVidia) and modify their age & gender & more with latent directions found by Robert Luxemburg ( https://twitter.com/robertluxemburg )

#After reading this notebook, you will be able to produce videos similar to these:

[![](http://img.youtube.com/vi/67wReX6C7VA/0.jpg)](http://www.youtube.com/watch?v=67wReX6C7VA "Video Title")
[![](http://img.youtube.com/vi/Ty0PpXT0iE4/0.jpg)](http://www.youtube.com/watch?v=Ty0PpXT0iE4 "Video Title")

###Read the instructions, modify the different options to your likings & run the cells!

Before running the setup cell, make sure you've selected a GPU runtime. Press runtime in the upper bar, click change runtime type & select GPU.

In [ ]:
#@title #Setup (approximate time: 2 minutes)
%tensorflow_version 1.x
!git clone https://github.com/justinpinkney/stylegan2

%cd stylegan2

import tensorflow as tf

import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
from io import BytesIO
import IPython.display
import numpy as np
from math import ceil
from PIL import Image, ImageDraw
import imageio

import pretrained_networks

def generate_images_in_w_space(dlatents, truncation_psi):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    Gs_kwargs.truncation_psi = truncation_psi
    dlatent_avg = Gs.get_var('dlatent_avg') # [component]
 
    imgs = []
    for row, dlatent in log_progress(enumerate(dlatents), name = "Generating images"):
        #row_dlatents = (dlatent[np.newaxis] - dlatent_avg) * np.reshape(truncation_psi, [-1, 1, 1]) + dlatent_avg
        dl = (dlatent-dlatent_avg)*truncation_psi   + dlatent_avg
        row_images = Gs.components.synthesis.run(dlatent,  **Gs_kwargs)
        imgs.append(PIL.Image.fromarray(row_images[0], 'RGB'))
    return imgs

def imshow(a, format='png', jpeg_fallback=True):
  a = np.asarray(a, dtype=np.uint8)
  str_file = BytesIO()
  PIL.Image.fromarray(a).save(str_file, format)
  im_data = str_file.getvalue()
  try:
    disp = IPython.display.display(IPython.display.Image(im_data))
  except IOError:
    if jpeg_fallback and format != 'jpeg':
      print ('Warning: image was too large to display in format "{}"; '
             'trying jpeg instead.').format(format)
      return imshow(a, format='jpeg')
    else:
      raise
  return disp

def createImageGrid(images, scale=0.25, rows=1):
   w,h = images[0].size
   w = int(w*scale)
   h = int(h*scale)
   height = rows*h
   cols = ceil(len(images) / rows)
   width = cols*w
   canvas = PIL.Image.new('RGBA', (width,height), 'white')
   for i,img in enumerate(images):
     img = img.resize((w,h), PIL.Image.ANTIALIAS)
     canvas.paste(img, (w*(i % cols), h*(i // cols))) 
   return canvas

def loadLatent(path):
  return np.expand_dims(np.load(path),axis=0)

def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

def interpolate(zs, steps):
   out = []
   for i in range(len(zs)-1):
    for index in range(steps):
     fraction = index/float(steps) 
     out.append(zs[i+1]*fraction + zs[i]*(1-fraction))
   return out
   
import pretrained_networks

network_pkl = "gdrive:networks/stylegan2-ffhq-config-f.pkl"
 
# If downloads fails, due to 'Google Drive download quota exceeded' you can try downloading manually from your own Google Drive account
# network_pkl = "/content/drive/My Drive/GAN/stylegan2-ffhq-config-f.pkl"
 
print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)
noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

!mkdir raw
!mkdir aligned
!mkdir output

print("Finished!")

#Upload your images
##Now upload your face image(s) to the raw folder in the stylegan2 directory. Press the folder icon on the left site to open the file browser if it isn't opened already.

##The images should contain a human face. Good lighting conditions, high resolution & a central perspective will help to improve the results.

###After you've done that, execute the next cell. It'll try to identify faces in the image(s) you uploaded & prepare them for the StyleGAN2 model & save the results in the aligned folder.

In [ ]:
#@title #Align Images. Download will sometimes get stuck, restarting likely helps. Alternatively, you can download & put the network in your drive & change the code in align_images to read the network from your drive.
!python align_images.py raw/ aligned/

#Next step: Projecting your aligned images
## In the following cell, you can adjust the parameter num-steps to your likings. A higher number will result in a longer execution time while it likely increases the quality of your output.
### At higher step sizes weird artifcats may arise as a result of the network desperately trying to generate features it technically can't.


#####Add "--video True --video-mode 2" to the arguments if you want to get a video of the projection process. It'll increase the runtime by a lot.

In [ ]:
!python project_images.py --num-steps 350 aligned generated

## After the previous cell has finished its execution you should find 2 files for each aligned image in stylegan2/generated.

##One is a .png that shows how the generated person looked like after projection. If you want to reuse the projection you have just done you should download the .npy file. With it you can always load the latent (= input vector) with which you can generate the face that you can see in the .png.



In [ ]:
npyPath = "/content/stylegan2/generated/berniesanders_01.npy" #Change this to fit the path of your file.
latent = loadLatent(npyPath)
imshow(generate_images_in_w_space([latent],1.0)[0])

##If you have created multiple latents, you can interpolate between them.

##Change steps to adjust the intermediate steps between two latents.

In [ ]:
steps = 10
fps = 25

movieOutPath = "interpolation.mp4"

latentA = loadLatent("/content/stylegan2/generated/elon musk_01.npy") #Change path according to your case
latentB = loadLatent(npyPath)

latents = [latentA,latentB] #You can also add more latents here

images = generate_images_in_w_space(interpolate(latents,steps),1.0)

with imageio.get_writer(movieOutPath, mode='I', fps= fps) as writer:
  for img in images:
    writer.append_data(np.array(img))

#To edit facial features like demonstrated in the videos of [this](https://www.youtube.com/playlist?list=PLDFtAMvjSK0Z88fSruugz8CZ4AqkZjDdZ) playlist, first download the directions.zip from [this](https://twitter.com/robertluxemburg/status/1207087801344372736) tweet. Upload them to the colab & unzip them using next the next cell.

In [ ]:
#Adjust path if necessary
!unzip /content/latent_directions.zip

In [ ]:
latent = loadLatent("/content/stylegan2/generated/obama_01 (1).npy") #Adjust this path to your case.
feature = loadLatent("/content/stylegan2/stylegan2directions/smile.npy") #Change the facial feature you want to edit here.

steps = 9 
#Negate the factor to go in the other direction
factor = 2

latents = []
for i in range(steps):
  latents.append(latent+feature*i*factor)

imgs = generate_images_in_w_space(latents,1.0)

import math
imshow(createImageGrid(imgs, scale=0.2, rows=int(math.sqrt(steps))))

##LMK if you're missing an important feature in this notebook or if you need help.
###Instead of creating an image grid you could also interpolate between the final edited latent & the original one. For that, simply copy & paste some code from 2 cells ago.

##If you liked this notebook I'd be pleased if you expressed your gratitude by dropping a sub on my [YT channel](https://www.youtube.com/channel/UCIkA_Pi0VWSABdyAnmildpg?sub_confirmation=1). If you're generally interested in AI & new computer technologies, you are likely to enjoy the channel [Two Minute Papers](https://www.youtube.com/user/keeroyzw).